In [20]:
%load_ext autoreload
%autoreload
import sys
import pickle
sys.path.append("../")
from mit_data import MITDataset
from mit_data import train_val_split
from mit_data import make_label_binarizer
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
original_df = pd.read_csv("../data/MIT_data/train_index.csv")
list_original = original_df.to_dict("records")
for data in list_original:
    data["object_label_list"] = str(data["object_label"]).split(" ")

In [6]:
def regularize_table(df):
    labels = df["object_label"].map(lambda x: str(x).split(" "))
    expanded = []
    # for each row in table
    for n in range(df.shape[0]):
        # for each label in list
        for i in range(len(labels[n])):
            dict_element = df.iloc[n].to_dict()
            dict_element["object_label"] = labels[n][i]
            dict_element["index"] = n
            expanded.append(dict_element)
    expanded_df = pd.DataFrame(expanded)
    expanded_df["object_label"] = expanded_df["object_label"].map(
        lambda x: np.nan if x == 'nan' else x)
    return expanded_df


In [10]:
expanded_df = regularize_table(original_df)

In [21]:
NUM_class = 20
label_count = expanded_df[["object_label", "index"]].groupby(
    ["object_label"], as_index=False).count()
sorted_df = label_count.sort_values("index", ascending=False)[:NUM_class]
classes = set(sorted_df["object_label"].values)
with open("../experiment/top_30_class/classes.pkl", "wb") as f:
    pickle.dump(classes,f)

In [22]:
list_original[0]

{'index': 0,
 'category': 'breaking',
 'directory': '../data/MIT_data/training/breaking',
 'filename': 'v018_0048.mp4',
 'train': 'train',
 'object_label': 'hand cigarette',
 'scene_label': nan,
 'object_label_list': ['hand', 'cigarette']}

In [31]:
list_with_top_category = list(filter(lambda data: len(classes & set(data["object_label_list"])) != 0 , list_original))
list_without_top_category =list(filter(lambda data: len(classes & set(data["object_label_list"])) == 0 , list_original))
#list_without_man_cvt = list_without_man[:len(list_with_man)]
for data in list_with_top_category:
    new_labels = classes & set(data["object_label_list"])
    data["object_label"] = " ".join(new_labels)
    print(data["object_label"])

hand
{'hand'}
snow
{'snow'}
man
{'man'}
man
{'man'}
man
{'man'}
baby
{'baby'}
woman
{'woman'}
woman
{'woman'}
woman
{'woman'}
man
{'man'}
hand
{'hand'}
man
{'man'}
hand
{'hand'}
baby
{'baby'}
road car
{'road', 'car'}
baby
{'baby'}
baby
{'baby'}
woman girl
{'woman', 'girl'}
woman
{'woman'}
man
{'man'}
woman
{'woman'}
boy
{'boy'}
man
{'man'}
woman
{'woman'}
man
{'man'}
man woman
{'man', 'woman'}
girl
{'girl'}
man
{'man'}
girl
{'girl'}
man baby
{'man', 'baby'}
woman
{'woman'}
woman
{'woman'}
woman boy
{'woman', 'boy'}
man
{'man'}
man woman
{'man', 'woman'}
man
{'man'}
man
{'man'}
woman girl
{'woman', 'girl'}
man boy
{'man', 'boy'}
girl
{'girl'}
girl
{'girl'}
man
{'man'}
hand
{'hand'}
woman
{'woman'}
man
{'man'}
man
{'man'}
man
{'man'}
man
{'man'}
man
{'man'}
grass
{'grass'}
water
{'water'}
water woman man
{'water', 'woman', 'man'}
water girl
{'water', 'girl'}
water woman
{'water', 'woman'}
water
{'water'}
water grass girl
{'water', 'grass', 'girl'}
water
{'water'}
grass
{'grass'}
dog
{'do

In [38]:
converted_df = pd.DataFrame(list_with_top_category)
converted_df = converted_df.drop(["object_label_list", "index"], axis=1)
converted_df.to_csv("../experiment/top_30_class/index.csv", index_label="index")

In [39]:
train_val_split(len(converted_df),save_file="../experiment/top_30_class/split.csv")

In [125]:
train_binary_dataset = MITDataset(mlb,mode="train",split_file="binary_split.csv",index_file="binary_label_man.csv")